<a href="https://colab.research.google.com/github/ytg000629/daegu_bigdata/blob/main/TEXTRANK%EB%AC%B8%EC%9E%A5%EC%9A%94%EC%95%BD_TTS_KEYBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install emoji
!pip install konlpy
!pip install google-cloud-texttospeech
!pip install google-cloud-speech
!pip install pydub
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 10.5 MB/s eta 0:00:00


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.0 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23777 sha256=b7afa682b0bd66ef389e6ea32880523fb395b79b87efa4956c47e84c19daf7fd
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04

In [55]:
dpath = "/content/drive/MyDrive/대구 빅데이터/output/"

In [4]:
import pandas as pd
import numpy as np
import emoji
import re
from tqdm import tqdm
from nltk.corpus import stopwords
import nltk
import string
import networkx
import re
from konlpy.tag import Komoran
from google.cloud import texttospeech
from google.cloud import speech
import os
import io
import warnings

warnings.filterwarnings("ignore")


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [56]:
data = pd.read_csv(dpath+'대구 패션.csv')

In [57]:
data.head(20)

,date,title,link,content
0,2023-06-22 13:12:06,대구염색산업단지 이전 타당성 용역 착수보고회,https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n【대구=뉴시스】대구염색산업단지 관리공단 전경. 뉴시스DB. 20...
1,2023-06-23 17:59:00,대구염색산업단지 이전 타당성 연구용역 착수,https://n.news.naver.com/mnews/article/002/000...,[\n이전 후보지 선정에 관심대구시가 23일 염색산업단지 이전 기본계획 수립 및 타...
2,2023-06-22 15:51:44,"""백캉스 오세요""...롯데백화점, 패션·골프 시즌오프 할인",https://n.news.naver.com/mnews/article/008/000...,[\n\n\n\n\n/사진제공=롯데백화점롯데백화점이 여름을 맞아 '시즌오프' 할인행...
3,2023-06-21 11:24:01,사라지는 계절감…여름에도 '니트' 잘 팔린다는데[최수진의 패션채널],https://n.news.naver.com/mnews/article/050/000...,"[\n겨울 제품에 사용되는 니트, 여름에도 인기 얻으며 제품화돼올여름에는 Y2K 트..."
4,2023-06-22 17:18:01,"디자인진흥원-대구시, 산업부 지역 퍼스널케어 산업 고도화 지원 나서",https://n.news.naver.com/mnews/article/088/000...,"[\n오는 7월 12일까지 참여기업 모집, 제품디자인, 브랜드, 콘테츠 등 분야별 ..."
5,2023-06-19 08:30:01,"삼성패션, 청담동에 유럽 풋웨어 '숄' 팝업스토어",https://n.news.naver.com/mnews/article/018/000...,[\n\t\t\t[이데일리 정병묵 기자] 삼성물산(028260) 패션부문이 운영하는...
6,2023-06-21 14:50:04,“해외 명품보다 잘 나가네!” K패션 르네상스 시대,https://n.news.naver.com/mnews/article/468/000...,[\n\n\n\n\n더현대서울 입점 3일만에 3억원이 넘는 매출고를 올린 마뗑킴. ...
7,2023-06-22 14:22:01,오페라 '춘향전'을 판소리 '춘향가'에 접목시켰다…레저 오페라 '춘향전-패션을 입다!',https://n.news.naver.com/mnews/article/088/000...,"[\n6월 28일 오후 7시 30분, 아양아트센터 아양홀에서\n\n\n\n레저오페라..."
8,2023-06-18 13:59:01,젊은이 패션성지 무신사·W컨셉 “옷 잘 입는 반쪽 찾아라”,https://n.news.naver.com/mnews/article/032/000...,[\n무신사 ‘29CM’ 인수로 여성 단골 대거 확보W컨셉 젊은 남성 브랜드 인기에...
9,2023-06-24 05:02:01,"주말엔 '백캉스' 떠나볼까, 백화점 여름 할인전 개시",https://n.news.naver.com/mnews/article/079/000...,[\n무더위 피해 쇼핑 선택하는 소비자 겨냥 대규모 할인 행사패션·뷰티에 와인 클래...


In [63]:
for i in range(len(data)):
    #data.content[i] = data.content[i].replace('\n','')
    #data.content[i] = data.content[i].replace('\t','')
    data.content[i] = data.content[i].replace('■','')
    data.content[i] = data.content[i].replace('▲','')
    data.content[i] = data.content[i].replace('com(재)','')


In [64]:
data.head(20)

,date,title,link,content
0,2023-06-22 13:12:06,대구염색산업단지 이전 타당성 용역 착수보고회,https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n【대구=뉴시스】대구염색산업단지 관리공단 전경. 뉴시스DB. 20...
1,2023-06-23 17:59:00,대구염색산업단지 이전 타당성 연구용역 착수,https://n.news.naver.com/mnews/article/002/000...,[\n이전 후보지 선정에 관심대구시가 23일 염색산업단지 이전 기본계획 수립 및 타...
2,2023-06-22 15:51:44,"""백캉스 오세요""...롯데백화점, 패션·골프 시즌오프 할인",https://n.news.naver.com/mnews/article/008/000...,[\n\n\n\n\n/사진제공=롯데백화점롯데백화점이 여름을 맞아 '시즌오프' 할인행...
3,2023-06-21 11:24:01,사라지는 계절감…여름에도 '니트' 잘 팔린다는데[최수진의 패션채널],https://n.news.naver.com/mnews/article/050/000...,"[\n겨울 제품에 사용되는 니트, 여름에도 인기 얻으며 제품화돼올여름에는 Y2K 트..."
4,2023-06-22 17:18:01,"디자인진흥원-대구시, 산업부 지역 퍼스널케어 산업 고도화 지원 나서",https://n.news.naver.com/mnews/article/088/000...,"[\n오는 7월 12일까지 참여기업 모집, 제품디자인, 브랜드, 콘테츠 등 분야별 ..."
5,2023-06-19 08:30:01,"삼성패션, 청담동에 유럽 풋웨어 '숄' 팝업스토어",https://n.news.naver.com/mnews/article/018/000...,[\n\t\t\t[이데일리 정병묵 기자] 삼성물산(028260) 패션부문이 운영하는...
6,2023-06-21 14:50:04,“해외 명품보다 잘 나가네!” K패션 르네상스 시대,https://n.news.naver.com/mnews/article/468/000...,[\n\n\n\n\n더현대서울 입점 3일만에 3억원이 넘는 매출고를 올린 마뗑킴. ...
7,2023-06-22 14:22:01,오페라 '춘향전'을 판소리 '춘향가'에 접목시켰다…레저 오페라 '춘향전-패션을 입다!',https://n.news.naver.com/mnews/article/088/000...,"[\n6월 28일 오후 7시 30분, 아양아트센터 아양홀에서\n\n\n\n레저오페라..."
8,2023-06-18 13:59:01,젊은이 패션성지 무신사·W컨셉 “옷 잘 입는 반쪽 찾아라”,https://n.news.naver.com/mnews/article/032/000...,[\n무신사 ‘29CM’ 인수로 여성 단골 대거 확보W컨셉 젊은 남성 브랜드 인기에...
9,2023-06-24 05:02:01,"주말엔 '백캉스' 떠나볼까, 백화점 여름 할인전 개시",https://n.news.naver.com/mnews/article/079/000...,[\n무더위 피해 쇼핑 선택하는 소비자 겨냥 대규모 할인 행사패션·뷰티에 와인 클래...


In [65]:
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')

    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')

    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0


    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1

        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1

        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])

    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))

        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)

        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s

    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])

    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])

    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))

    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')

    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi

        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)

        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)

        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)

        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)

        return both

    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

In [51]:
from keybert import KeyBERT
kw_model = KeyBERT()

In [72]:
result = []
num = []
no_result = []

for i in range(len(data)):
    f = open(f"content.txt{i}", 'w')
    f.write(data.content[i])
    f.close()
    tr = TextRank()
    tagger = Komoran()
    stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
    tr.loadSents(RawSentenceReader(f'content.txt{i}'), lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))
    tr.build()
    ranks = tr.rank()
    #for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    #print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
    #print("\t".join([str(tr.dictCount[k])]))
    a = tr.summarize(0.2)
    if a == '':
        #print(f'{i}.' ,'핵심 문장이 발견X')
        #a = '핵심 문장이 발견되지 않았습니다. 링크를 눌러 기사 원문을 참고해주십시오'
        no_result.append(a)
    elif len(a) > 250:
        #print(f'{i}.' ,'너무 길어')
        no_result.append(a)
    else:
        #print(f'{i}.' , a)
        result.append(a)
        num.append(i)
    #print('')
    if len(result) == 3:
        break

print('키워드 추출')

for i,j in enumerate(num):
    doc = data.content[j]
    keywords_mmr = kw_model.extract_keywords(doc,keyphrase_ngram_range=(2,4),use_mmr = True,top_n = 3,diversity = 0.3)
    print(f'{i+1}.' ,keywords_mmr)

print('')

print('핵심문장요약')

for i in range(3):
    print(f'{i+1}.' , result[i])

키워드 추출
1. [('염색산업의 경쟁력 제고와', 0.7045), ('용역은 염색산업단지의', 0.704), ('하고 있는 대구정책연구원과 염색가공', 0.7002)]
2. [('공무원 김이진 대구염색산업단지관리공단 이사장', 0.6688), ('하고 있는 대구정책연구원과 염색가공', 0.6629), ('수립한다는 계획이다 용역은 염색산업단지의', 0.6617)]
3. [('육성과 지역 제조기업의 활력을', 0.6595), ('상품성 확대 제조 활성화', 0.6029), ('기자 woon imaeil 대구경북디자인진흥원이', 0.5107)]

핵심문장요약
1.  최종 용역 결과를 반영해 내년 하반기에 염색산단 이전지를 선정하고 대구염색산업단지 이전 기본계획을 수립한다는 계획이다.
2. 이전 후보지 선정에 관심대구시가 23일 염색산업단지 이전 기본계획 수립 및 타당성 검토 연구용역’ 착수보고회를 개최했다.  23일지역 섬유·염색산업의 경쟁력 제고와 탄소중립 첨단산업단지로의 전환을 위한 ‘대구 염색산업단지 이전 기본계획 수립 및 타당성 검토 연구용역’ 착수보고회를 개최했다.
3. 대구경북디자인진흥원이 대구시, 산업통상부와 함께 생활소비재(헬스, 리빙, 뷰티케어) 산업의 상품성 확대 및 제조 활성화 지원하는 '퍼스널케어 융합 얼라이언스 육성사업'을 본격 추진한다고 22일 밝혔다.


In [ ]:
#Google Cloud Text To Speech(TTS) API
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/drive/MyDrive/대구 빅데이터/daegubigdata-d6b45b2dabc8.json"

def TextToSpeech(text, file_name):
    client = texttospeech.TextToSpeechClient()
    input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(language_code = 'ko-KR', ssml_gender = texttospeech.SsmlVoiceGender.FEMALE)
    audio_config = texttospeech.AudioConfig(audio_encoding = texttospeech.AudioEncoding.LINEAR16)
    response = client.synthesize_speech(input = input, voice = voice, audio_config = audio_config)

    with open(file_name, 'wb') as out:
        out.write(response.audio_content)

In [ ]:
#Colab 오디오를 통해 듣기
from IPython.display import Audio

TextToSpeech('外', "./Answer.wav")
audio_file = '/content/Answer.wav'
Audio(audio_file)